In [11]:
import numpy as np
import requests
from io import StringIO
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import Callback
import tensorflow as tf

class PrintMetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}/{self.params['epochs']} - loss: {logs['loss']:.4f} - accuracy: {logs['accuracy']:.4f} - val_loss: {logs['val_loss']:.4f} - val_accuracy: {logs['val_accuracy']:.4f}")

def load_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra"
    response = requests.get(url)
    data = np.genfromtxt(StringIO(response.text), delimiter=',')
    np.random.shuffle(data)
    X = data[:, :-1]
    y = data[:, -1].astype(int)
    X /= 16.0
    return X, y

X, y = load_data()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the neural network model
model = Sequential()
model.add(Dense(100, input_shape=(X_train.shape[1],), activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the callback for printing metrics
print_metrics_callback = PrintMetricsCallback()

# Train the model with the callback
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0, callbacks=[print_metrics_callback])

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"\nFinal Train Accuracy: {train_accuracy*100:.2f}%, Final Train Loss: {train_loss:.4f}")
print(f"Final Test Accuracy: {test_accuracy*100:.2f}%, Final Test Loss: {test_loss:.4f}")

# Make predictions on test examples
@tf.function
def predict_examples(X_test):
    return model(X_test)

predicted_probs = predict_examples(X_test)
predicted_labels = np.argmax(predicted_probs, axis=1)

# Show predicted and actual labels for a few examples
num_examples = 5
random_indices = np.random.choice(X_test.shape[0], num_examples, replace=False)
X_examples = X_test[random_indices]
y_examples = y_test[random_indices]

for i in range(num_examples):
    print(f"\nExample {i+1}: Predicted Label: {predicted_labels[random_indices[i]]}, Actual Label: {np.argmax(y_examples[i])}")


Epoch 1/10 - loss: 2.2451 - accuracy: 0.2368 - val_loss: 2.1124 - val_accuracy: 0.4693
Epoch 2/10 - loss: 1.8990 - accuracy: 0.5883 - val_loss: 1.5944 - val_accuracy: 0.7229
Epoch 3/10 - loss: 1.3478 - accuracy: 0.7508 - val_loss: 1.0627 - val_accuracy: 0.7791
Epoch 4/10 - loss: 0.9144 - accuracy: 0.8489 - val_loss: 0.7401 - val_accuracy: 0.8758
Epoch 5/10 - loss: 0.6605 - accuracy: 0.8833 - val_loss: 0.5370 - val_accuracy: 0.9150
Epoch 6/10 - loss: 0.4980 - accuracy: 0.9124 - val_loss: 0.4235 - val_accuracy: 0.9190
Epoch 7/10 - loss: 0.3987 - accuracy: 0.9215 - val_loss: 0.3338 - val_accuracy: 0.9346
Epoch 8/10 - loss: 0.3251 - accuracy: 0.9346 - val_loss: 0.2878 - val_accuracy: 0.9399
Epoch 9/10 - loss: 0.2776 - accuracy: 0.9418 - val_loss: 0.2444 - val_accuracy: 0.9477
Epoch 10/10 - loss: 0.2424 - accuracy: 0.9483 - val_loss: 0.2116 - val_accuracy: 0.9621

Final Train Accuracy: 94.77%, Final Train Loss: 0.2258
Final Test Accuracy: 96.21%, Final Test Loss: 0.2116

Example 1: Predicte